[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/82_pmtiles.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/82_pmtiles.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)


**Visualizing PMTiles with leafmap**

[PMTiles](https://github.com/protomaps/PMTiles) is a single-file archive format for tiled data. A PMTiles archive can be hosted on a commodity storage platform such as S3, and enables low-cost, zero-maintenance map applications that are "serverless" - free of a custom tile backend or third party provider.

The PMTiles functionality for leafmap is based on the [folium-pmtiles](https://github.com/jtmiclat/folium-pmtiles) package. Credits to [Jt Miclat](https://github.com/jtmiclat).

In [ ]:
# %pip install -U leafmap pmtiles

In [ ]:
import leafmap.foliumap as leafmap

## Remote PMTiles

### Protomaps sample data

In [ ]:
url = "https://open.gishub.org/data/pmtiles/protomaps_firenze.pmtiles"
metadata = leafmap.pmtiles_metadata(url)
print(f"layer names: {metadata['layer_names']}")
print(f"bounds: {metadata['bounds']}")

In [ ]:
m = leafmap.Map()

style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "buildings",
            "source": "example_source",
            "source-layer": "landuse",
            "type": "fill",
            "paint": {"fill-color": "steelblue"},
        },
        {
            "id": "roads",
            "source": "example_source",
            "source-layer": "roads",
            "type": "line",
            "paint": {"line-color": "black"},
        },
    ],
}

# style = leafmap.pmtiles_style(url)  # Use default style

m.add_pmtiles(
    url, name="PMTiles", style=style, overlay=True, show=True, zoom_to_layer=True
)
m

### Source Cooperative

[Google-Microsoft Open Buildings - combined by VIDA](https://beta.source.coop/repositories/vida/google-microsoft-open-buildings/description)

In [ ]:
url = "https://data.source.coop/vida/google-microsoft-open-buildings/pmtiles/go_ms_building_footprints.pmtiles"
metadata = leafmap.pmtiles_metadata(url)
print(f"layer names: {metadata['layer_names']}")
print(f"bounds: {metadata['bounds']}")

In [ ]:
m = leafmap.Map(center=[20, 0], zoom=2, height="800px")
m.add_basemap("CartoDB.DarkMatter")
m.add_basemap("Esri.WorldImagery", show=False)

style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "buildings",
            "source": "example_source",
            "source-layer": "building_footprints",
            "type": "fill",
            "paint": {"fill-color": "#3388ff", "fill-opacity": 0.5},
        },
    ],
}

# style = leafmap.pmtiles_style(url)  # Use default style

m.add_pmtiles(
    url, name="Buildings", style=style, overlay=True, show=True, zoom_to_layer=False
)

html = "Source: <a href='https://beta.source.coop/repositories/vida/google-microsoft-open-buildings/description' target='_blank'>source.coop</a>"
m.add_html(html, position="bottomright")

m

In [ ]:
m.save("buildings.html")

## Local PMTiles

tippecanoe is required to convert vector data to pmtiles. Install it with `conda install -c conda-forge tippecanoe`.

Download [building footprints](https://github.com/opengeos/open-data/blob/main/datasets/libya/Derna_buildings.geojson) of Derna, Libya.

In [ ]:
url = "https://raw.githubusercontent.com/opengeos/open-data/main/datasets/libya/Derna_buildings.geojson"
leafmap.download_file(url, "buildings.geojson")

Convert vector to PMTiles.

In [ ]:
pmtiles = "buildings.pmtiles"
leafmap.geojson_to_pmtiles(
    "buildings.geojson", pmtiles, layer_name="buildings", overwrite=True, quiet=True
)

Start a HTTP Sever

In [ ]:
leafmap.start_server(port=8000)

In [ ]:
url = f"http://127.0.0.1:8000/{pmtiles}"
# leafmap.pmtiles_metadata(url)

Display the PMTiles on the map.

In [ ]:
m = leafmap.Map()

style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "buildings",
            "source": "example_source",
            "source-layer": "buildings",
            "type": "fill",
            "paint": {"fill-color": "#3388ff", "fill-opacity": 0.5},
        },
    ],
}

# style = leafmap.pmtiles_style(url)  # Use default style

m.add_pmtiles(url, name="Buildings", show=True, zoom_to_layer=True, style=style)
m